# Silence Culture Analysis Project
## ISSP 2022 - Family and Changing Gender Roles Survey

This notebook analyzes public attitudes on sensitive social topics including health beliefs, gender roles, family structures, and personal information sharing patterns.

### Import Required Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
import statsmodels.api as sm
from scipy import stats

# Suppress warnings for cleaner output
warnings.filterwarnings('ignore')

# Set plotting style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

### Load Data
Load the CSV file containing survey responses

In [6]:
# Load the survey dataset
df = pd.read_stata('2022/2022Data.dta')

---
# PART 1: Global Research on "Culture of Silence"
## Analyzing the "Silence Hierarchy"

### 1.1 Define Topic Categories

We group survey questions into thematic categories to measure which topics people avoid discussing (refusal, "cannot decide", or no response). Higher "silence rate" indicates greater social taboo.

In [ ]:
# Define question groups by thematic category
# Each category contains question that address related topics
# We'll calculate a silence score for each category

categories_mapping = {
    'Sensitive Personal Info': ['q143', 'q144', 'q145', 'q146', 'q147', 'q173', 'q174', 'q177', 'q178', 'q179', 'q180', 'q181', 'q182'],
    'Gender Perceptions': ['q63', 'q64', 'q65', 'q66', 'q67', 'q68'],
    'Family Life Reality': ['q103', 'q104', 'q105', 'q106', 'q107', 'q108', 'q109', 'q110', 'q111', 'q112', 'q113'],
    'New Families & Morality': ['q77', 'q78', 'q79', 'q80', 'q81'],
    'Health System Trust': ['q3', 'q11', 'q12', 'q13', 'q14', 'q21', 'q22', 'q23'],
    'Corona Policy & Emergency': ['q57', 'q58', 'q59', 'q60', 'q61'],
    'Personal & Mental Health': ['q2', 'q37', 'q38', 'q39', 'q49', 'q50', 'q51', 'q52', 'q53', 'q54', 'q120'],
    'Digital Literacy & Health': ['q24', 'q25', 'q26', 'q27', 'q28', 'q29', 'q30', 'q31', 'q32'],
    'Family Solidarity': ['q115', 'q116', 'q117']
}

print("Topic Categories Defined:")
for category, questions in categories_mapping.items():
    print(f"  {category}: {len(questions)} questions")

### 1.2 Calculate Silence Scores

For each category, we calculate the percentage of people who:
- Refused to answer
- Selected "Cannot decide"
- Left the response blank (NaN)

This gives us a "silence score" that indicates how taboo a topic is - higher scores mean fewer people are willing to engage with the topic.

In [ ]:
# Function to calculate silence score for a group of questions
def calculate_silence_score(df, question_columns, refuse_keywords=['refuse', 'cannot choose', 'cannot decide']):
    """
    Calculate percentage of non-responses (refusal, cannot decide, NaN) for a set of questions.
    
    Parameters:
    - df: dataframe
    - question_columns: list of column names
    - refuse_keywords: list of keywords indicating refusal/inability to respond
    
    Returns: float (percentage of silence)
    """
    
    total_responses = 0
    silence_count = 0
    
    for col in question_columns:
        if col in df.columns:
            # Count missing values
            missing = df[col].isna().sum()
            silence_count += missing
            
            # Count refusal/cannot decide responses
            for keyword in refuse_keywords:
                silence_count += (df[col].astype(str).str.lower().str.contains(keyword, na=False)).sum()
            
            total_responses += len(df[col])
    
    # Calculate percentage of silence
    silence_percentage = (silence_count / total_responses * 100) if total_responses > 0 else 0
    return silence_percentage

# Calculate silence scores for each category
silence_scores = {}
for category, questions in categories_mapping.items():
    silence_scores[category] = calculate_silence_score(df, questions)

# Convert to DataFrame for easier visualization
silence_df = pd.DataFrame(list(silence_scores.items()), columns=['Topic', 'Silence Score'])
silence_df = silence_df.sort_values('Silence Score', ascending=False)

print("\n" + "="*60)
print("SILENCE HIERARCHY: Topics Ranked by Avoidance")
print("="*60)
print(silence_df.to_string(index=False))
print("\nInterpretation:")
print("- Higher scores = more taboo (people avoid discussing)")
print("- Lower scores = easier to discuss (more people engage)")

### 1.3 Visualize the Silence Hierarchy

A bar plot shows which social topics are most and least taboo in the studied population.

In [ ]:
# Create bar plot of silence scores
fig, ax = plt.subplots(figsize=(12, 7))

# Sort for better visualization (descending order)
silence_df_sorted = silence_df.sort_values('Silence Score', ascending=True)

# Create horizontal bar chart
bars = ax.barh(silence_df_sorted['Topic'], silence_df_sorted['Silence Score'], 
                color=plt.cm.RdYlGn_r(silence_df_sorted['Silence Score']/silence_df_sorted['Silence Score'].max()))

# Customize plot
ax.set_xlabel('Silence Score (% of Non-Responses)', fontsize=12, fontweight='bold')
ax.set_ylabel('Topic Category', fontsize=12, fontweight='bold')
ax.set_title('Silence Hierarchy: Which Topics Are Most Taboo?', fontsize=14, fontweight='bold', pad=20)
ax.set_xlim(0, silence_df['Silence Score'].max() * 1.1)

# Add value labels on bars
for i, (topic, score) in enumerate(zip(silence_df_sorted['Topic'], silence_df_sorted['Silence Score'])):
    ax.text(score + 0.5, i, f'{score:.1f}%', va='center', fontsize=10, fontweight='bold')

plt.tight_layout()
plt.show()

print("\nKey Insights:")
print(f"  Most Taboo: {silence_df.iloc[0]['Topic']} ({silence_df.iloc[0]['Silence Score']:.1f}% silence)")
print(f"  Least Taboo: {silence_df.iloc[-1]['Topic']} ({silence_df.iloc[-1]['Silence Score']:.1f}% silence)")
print(f"  Average Silence: {silence_df['Silence Score'].mean():.1f}%")

### 1.4 Descriptive Statistics on Silence Scores

In [ ]:
# Calculate and display descriptive statistics
print("\nDescriptive Statistics: Silence Scores Across All Topics")
print("="*60)
print(silence_df['Silence Score'].describe())
print("\nStatistical Interpretation:")
print(f"  - Mean (average): {silence_df['Silence Score'].mean():.2f}%")
print(f"  - Median (middle value): {silence_df['Silence Score'].median():.2f}%")
print(f"  - Std Dev (variability): {silence_df['Silence Score'].std():.2f}%")
print(f"  - Range: {silence_df['Silence Score'].min():.2f}% to {silence_df['Silence Score'].max():.2f}%")
print("\nWhat this means:")
print("  - High standard deviation suggests topics vary widely in how taboo they are")
print("  - Some topics are universally discussed while others are heavily avoided")

---
# PART 2: "Gender Silence": Where Is It Hardest to Talk About Women's Roles?

We analyze whether countries with traditional characteristics show:
- Lower variance (everyone agrees - conformity)
- Higher refusal rates (higher silence despite agreement)

### 2.1 Focus on Gender Role Questions (Q63, Q68)

These questions directly address gender role expectations:
- Q63: "Can working mothers maintain warm relationships with their children?"
- Q68: "Man's role is earning; woman's role is home and family"

In [ ]:
# Identify country column (adjust name if different in your dataset)
country_column = 'country'  # IMPORTANT: Change this to your actual country column name

# Define the gender role questions
gender_questions = ['q63', 'q68']  # IMPORTANT: Adjust these to your actual column names

# Verify these columns exist in the dataset
existing_columns = [col for col in gender_questions if col in df.columns]
print(f"Gender role questions found in dataset: {existing_columns}")

# Filter dataset for countries (remove missing values)
if country_column in df.columns:
    df_countries = df[df[country_column].notna()].copy()
    print(f"\nCountries in dataset: {df_countries[country_column].unique()}")
else:
    print(f"Warning: '{country_column}' not found. Using all data without country grouping.")
    df_countries = df.copy()

### 2.2 Prepare Data: One-Hot Encoding for Countries

We convert categorical country variable into numerical format (dummy variables) to enable analysis.

In [ ]:
# Function to calculate refusal/avoidance rate for a specific column
def calculate_refusal_rate(series, refuse_keywords=['refuse', 'cannot choose', 'cannot decide']):
    """
    Calculate percentage of refusals and non-responses in a series.
    
    Returns: float (percentage)
    """
    total = len(series.dropna())
    refusals = 0
    
    # Count missing values
    refusals += series.isna().sum()
    
    # Count refusal keywords
    for keyword in refuse_keywords:
        refusals += (series.astype(str).str.lower().str.contains(keyword, na=False)).sum()
    
    refusal_percentage = (refusals / len(series) * 100) if len(series) > 0 else 0
    return refusal_percentage

# Calculate refusal rates by country for gender questions
if country_column in df.columns:
    country_refusals = {}
    
    for country in df_countries[country_column].unique():
        country_data = df_countries[df_countries[country_column] == country]
        
        # Calculate average refusal rate across gender questions
        refusal_rates = [calculate_refusal_rate(country_data[col]) for col in existing_columns if col in country_data.columns]
        avg_refusal = np.mean(refusal_rates) if refusal_rates else 0
        country_refusals[country] = avg_refusal
    
    # Convert to DataFrame
    country_refusal_df = pd.DataFrame(list(country_refusals.items()), 
                                      columns=['Country', 'Avoidance Rate (%)'])
    country_refusal_df = country_refusal_df.sort_values('Avoidance Rate (%)', ascending=False)
    
    print("\n" + "="*60)
    print("GENDER SILENCE BY COUNTRY")
    print("Percentage of people avoiding gender role questions")
    print("="*60)
    print(country_refusal_df.to_string(index=False))
    else:
        print("\nCountry variable not found. Skipping country-level analysis.")

### 2.3 Analyze Response Variation Within Countries

We compare variance in responses (standard deviation) to understand whether some countries show conformity vs. disagreement.

In [ ]:
# Helper function to convert text responses to numerical values for analysis
def convert_likert_to_numeric(value):
    """
    Convert Likert-scale responses to numerical values.
    This helps us calculate mean and standard deviation.
    
"""
    likert_mapping = {
        'strongly agree': 2,
        'agree': 1,
        'neither agree nor disagree': 0,
        'disagree': -1,
        'strongly disagree': -2,
        'cannot decide': np.nan,
        'refuse': np.nan
    }
    
    if isinstance(value, str):
        for key, numeric_value in likert_mapping.items():
            if key.lower() in value.lower():
                return numeric_value
    return np.nan

# Create numeric versions of gender questions
for col in existing_columns:
    if col in df.columns:
        numeric_col = f'{col}_numeric'
        df[numeric_col] = df[col].apply(convert_likert_to_numeric)

# Analyze variance by country
if country_column in df.columns:
    variance_analysis = []
    
    for country in df_countries[country_column].unique():
        country_data = df_countries[df_countries[country_column] == country]
        
        # Get numeric versions and calculate statistics
        numeric_cols_existing = [f'{col}_numeric' for col in existing_columns if f'{col}_numeric' in country_data.columns]
        
        if numeric_cols_existing:
            # Calculate mean and std across gender questions
            means = []
            stds = []
            
            for col in numeric_cols_existing:
                col_data = country_data[col].dropna()
                if len(col_data) > 0:
                    means.append(col_data.mean())
                    stds.append(col_data.std())
            
            if means and stds:
                avg_mean = np.mean(means)
                avg_std = np.mean(stds)
                variance_analysis.append({
                    'Country': country,
                    'Mean Response': avg_mean,
                    'Std Dev (Disagreement)': avg_std
                })
    
    variance_df = pd.DataFrame(variance_analysis)
    variance_df = variance_df.sort_values('Std Dev (Disagreement)', ascending=False)
    
    print("\n" + "="*80)
    print("RESPONSE VARIATION IN GENDER QUESTIONS BY COUNTRY")
    print("Higher Std Dev = More Disagreement (heterogeneous views)")
    print("Lower Std Dev = More Conformity (homogeneous views)")
    print("="*80)
    print(variance_df.to_string(index=False))

### 2.4 Visualization: Gender Silence Across Countries

In [ ]:
# Create visualization of gender silence by country
if country_column in df.columns and not country_refusal_df.empty:
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))
    
    # Plot 1: Avoidance Rates
    ax1.bar(country_refusal_df['Country'], country_refusal_df['Avoidance Rate (%)'], 
            color='steelblue', alpha=0.7, edgecolor='black')
    ax1.set_xlabel('Country', fontsize=11, fontweight='bold')
    ax1.set_ylabel('Avoidance Rate (%)', fontsize=11, fontweight='bold')
    ax1.set_title('Gender Role Questions: Who Avoids Answering?', fontsize=12, fontweight='bold')
    ax1.tick_params(axis='x', rotation=45)
    
    # Plot 2: Response Variation
    if not variance_df.empty:
        ax2.scatter(variance_df['Mean Response'], variance_df['Std Dev (Disagreement)'], 
                   s=200, alpha=0.6, c=range(len(variance_df)), cmap='viridis', edgecolor='black')
        
        for idx, row in variance_df.iterrows():
            ax2.annotate(row['Country'], 
                        (row['Mean Response'], row['Std Dev (Disagreement)']),
                        xytext=(5, 5), textcoords='offset points', fontsize=9)
        
        ax2.set_xlabel('Average Response (Traditional → Liberal)', fontsize=11, fontweight='bold')
        ax2.set_ylabel('Standard Deviation (Disagreement)', fontsize=11, fontweight='bold')
        ax2.set_title('Homogeneity vs Disagreement in Gender Views', fontsize=12, fontweight='bold')
        ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
else:
    print("\nCountry-level visualization not possible without country column.")

### 2.5 Key Insights from Gender Silence Analysis

In [ ]:
print("\n" + "="*60)
print("GENDER SILENCE ANALYSIS: KEY FINDINGS")
print("="*60)

if not country_refusal_df.empty:
    max_avoidance = country_refusal_df.iloc[0]
    min_avoidance = country_refusal_df.iloc[-1]
    print(f"\n1. Highest Avoidance: {max_avoidance['Country']} ({max_avoidance['Avoidance Rate (%)']:.1f}%)")
    print(f"   Interpretation: People in this country are most reluctant to discuss gender roles")
    print(f"\n2. Lowest Avoidance: {min_avoidance['Country']} ({min_avoidance['Avoidance Rate (%)']:.1f}%)")
    print(f"   Interpretation: People in this country more openly discuss gender roles")

if not variance_df.empty:
    high_var = variance_df.iloc[0]
    low_var = variance_df.iloc[-1]
    print(f"\n3. Highest Disagreement: {high_var['Country']} (Std Dev: {high_var['Std Dev (Disagreement)']:.2f})")
    print(f"   Interpretation: People hold diverse views on gender roles (heterogeneous)")
    print(f"\n4. Highest Conformity: {low_var['Country']} (Std Dev: {low_var['Std Dev (Disagreement)']:.2f})")
    print(f"   Interpretation: People largely agree (homogeneous views)")

print("\n5. Paradox Analysis:")
print("   Do traditional countries show:")
print("   - High avoidance (silence) BUT high conformity (agreement)?")
print("   - High avoidance (silence) AND high disagreement (conflict)?")
print("   Different patterns suggest different social dynamics.")

---
# PART 3: "Economic Privacy Paradox": Predicting Income Question Refusers

Can we predict who will refuse to answer income questions (Q173-Q174) based on education, age, and home ownership?

### 3.1 Prepare Target Variable: Income Question Refusal

Create a binary variable:
- 1 = Refused to answer income questions ("cannot decide", "refuse", or missing)
- 0 = Provided an income response

In [ ]:
# Define income questions
income_questions = ['q173', 'q174']  # IMPORTANT: Adjust to your actual column names

# Create binary target: Did respondent refuse income questions?
def is_income_refusal(row):
    """
    Return 1 if person refused or cannot decide on BOTH income questions.
    Return 0 if person answered at least one income question.
    """
    refuse_keywords = ['refuse', 'cannot choose', 'cannot decide']
    
    refusals = 0
    total_questions = 0
    
    for col in income_questions:
        if col in df.columns:
            total_questions += 1
            value = row[col]
            
            # Check if missing or contains refusal keyword
            if pd.isna(value):
                refusals += 1
            elif any(keyword.lower() in str(value).lower() for keyword in refuse_keywords):
                refusals += 1
    
    # Return 1 if refused both questions, 0 otherwise
    return 1 if refusals == total_questions and total_questions > 0 else 0

# Apply function to create target variable
df['income_refusal'] = df.apply(is_income_refusal, axis=1)

# Check the distribution
refusal_counts = df['income_refusal'].value_counts()
print("\n" + "="*60)
print("INCOME QUESTION REFUSAL DISTRIBUTION")
print("="*60)
print(f"Answered income questions: {refusal_counts[0] if 0 in refusal_counts.index else 0} ({refusal_counts[0]/len(df)*100 if 0 in refusal_counts.index else 0:.1f}%)")
print(f"Refused income questions:  {refusal_counts[1] if 1 in refusal_counts.index else 0} ({refusal_counts[1]/len(df)*100 if 1 in refusal_counts.index else 0:.1f}%)")
print(f"\nTotal respondents: {len(df)}")

### 3.2 Prepare Predictor Variables

We'll use:
- Age (numerical)
- Education level (categorical → one-hot encoded)
- Home ownership (binary)

In [ ]:
# Define predictor variables (adjust column names as needed)
age_column = 'q127'  # IMPORTANT: Change if your age column has different name
education_column = 'q129'  # IMPORTANT: Change if your education column has different name
home_ownership_column = 'q155'  # IMPORTANT: Adjust based on your data

# Create working dataset for modeling
df_model = df.copy()

# 1. Prepare Age variable
if age_column in df_model.columns:
    # Convert birth year to age (or use age directly if already age)
    try:
        df_model['age'] = pd.to_numeric(df_model[age_column], errors='coerce')
        # If values are years of birth, convert to age
        if df_model['age'].max() > 150:  # Likely birth years
            current_year = 2024
            df_model['age'] = current_year - df_model['age']
    except:
        print(f"Warning: Could not process {age_column} as age")

# 2. Prepare Education variable
if education_column in df_model.columns:
    df_model['education'] = df_model[education_column].astype(str)
    # Count education categories
    print(f"\nEducation categories: {df_model['education'].nunique()}")
    print(df_model['education'].value_counts().head())
else:
    print(f"Warning: {education_column} not found")

# 3. Prepare Home Ownership variable
if home_ownership_column in df_model.columns:
    df_model['home_owner'] = df_model[home_ownership_column].notna().astype(int)
else:
    print(f"Warning: {home_ownership_column} not found")
    # Create dummy home ownership for demonstration
    df_model['home_owner'] = np.random.choice([0, 1], size=len(df_model))

print("\n" + "="*60)
print("PREDICTOR VARIABLES PREPARED")
print("="*60)
print(f"Age: {df_model['age'].describe().to_string()}")
print(f"\nHome Ownership Distribution:")
print(df_model['home_owner'].value_counts())

### 3.3 Feature Engineering and Data Cleaning

Prepare data for machine learning models by handling missing values and creating dummy variables.

In [ ]:
# Select features for modeling
feature_cols = ['age', 'education', 'home_owner']
df_clean = df_model[feature_cols + ['income_refusal']].copy()

# Remove rows with missing target variable
df_clean = df_clean[df_clean['income_refusal'].notna()]

# Handle missing values in predictors
print("Missing values before cleaning:")
print(df_clean.isnull().sum())

# Drop rows with missing age (critical predictor)
df_clean = df_clean[df_clean['age'].notna()]

# Fill education with 'Unknown' if missing
df_clean['education'].fillna('Unknown', inplace=True)

# Fill home_owner with mode (0) if missing
df_clean['home_owner'].fillna(0, inplace=True)

print("\nMissing values after cleaning:")
print(df_clean.isnull().sum())
print(f"\nDataset shape for modeling: {df_clean.shape}")

### 3.4 One-Hot Encoding for Education

Convert the categorical education variable into binary dummy variables for the logistic regression model.

In [ ]:
# Create dummy variables for education
# drop_first=True avoids multicollinearity (perfect linear relationship)
X_education_dummies = pd.get_dummies(df_clean[['education']], drop_first=True)

print("Education dummy variables created:")
print(X_education_dummies.head())
print(f"\nShape: {X_education_dummies.shape}")

# Combine age, home ownership, and education dummies
X = pd.concat([df_clean[['age', 'home_owner']], X_education_dummies], axis=1)
y = df_clean['income_refusal']

print(f"\nFinal predictor variables:")
print(X.columns.tolist())
print(f"Predictor matrix shape: {X.shape}")
print(f"Target variable distribution:\n{y.value_counts()}")

### 3.5 Feature Scaling

Standardize predictors so each has mean=0 and std=1. This is important for logistic regression and KNN.

In [ ]:
# Standardize features (important for logistic regression and KNN)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_scaled_df = pd.DataFrame(X_scaled, columns=X.columns)

print("Feature scaling results:")
print(f"\nMean of scaled features (should be ~0):")
print(X_scaled_df.mean())
print(f"\nStandard deviation of scaled features (should be ~1):")
print(X_scaled_df.std())
print("\nScaling ensures all features contribute equally to the model.")

### 3.6 Train-Test Split

Divide data into training (80%) and test (20%) sets to evaluate model performance.

In [ ]:
# Split data: 80% training, 20% testing
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled_df, y, test_size=0.2, random_state=42, stratify=y
)

print("\n" + "="*60)
print("TRAIN-TEST SPLIT SUMMARY")
print("="*60)
print(f"Training set size: {len(X_train)} ({len(X_train)/len(X_scaled_df)*100:.1f}%)")
print(f"Test set size: {len(X_test)} ({len(X_test)/len(X_scaled_df)*100:.1f}%)")
print(f"\nTarget variable distribution in training set:")
print(y_train.value_counts())
print(f"\nTarget variable distribution in test set:")
print(y_test.value_counts())
print("\nNote: Stratified split ensures both sets have similar target distributions.")

### 3.7 Build Logistic Regression Model

Logistic regression predicts probability of income refusal. It's ideal for binary classification.

In [ ]:
# Train logistic regression model
log_reg = LogisticRegression(random_state=42, max_iter=1000)
log_reg.fit(X_train, y_train)

# Make predictions on test set
y_pred_log = log_reg.predict(X_test)
y_pred_proba = log_reg.predict_proba(X_test)

# Calculate accuracy
accuracy_log = accuracy_score(y_test, y_pred_log)

print("\n" + "="*60)
print("LOGISTIC REGRESSION: INCOME REFUSAL PREDICTION")
print("="*60)
print(f"\nModel Accuracy: {accuracy_log:.3f} ({accuracy_log*100:.1f}%)")
print(f"Interpretation: The model correctly predicts refusal {accuracy_log*100:.1f}% of the time.")

# Display model coefficients
print(f"\nModel Coefficients (interpretation):")
for feature, coef in zip(X.columns, log_reg.coef_[0]):
    print(f"  {feature}: {coef:.4f}")
    if coef > 0:
        print(f"    ↑ Increases probability of refusing income questions")
    elif coef < 0:
        print(f"    ↓ Decreases probability of refusing income questions")

print(f"\nIntercept (baseline): {log_reg.intercept_[0]:.4f}")

### 3.8 Build K-Nearest Neighbors (KNN) Model

KNN is a non-linear classifier that works well for comparison with logistic regression.

In [ ]:
# Train KNN model with k=5 neighbors
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)

# Make predictions
y_pred_knn = knn.predict(X_test)

# Calculate accuracy
accuracy_knn = accuracy_score(y_test, y_pred_knn)

print("\n" + "="*60)
print("K-NEAREST NEIGHBORS (KNN, k=5): INCOME REFUSAL PREDICTION")
print("="*60)
print(f"\nModel Accuracy: {accuracy_knn:.3f} ({accuracy_knn*100:.1f}%)")
print(f"\nComparison with Logistic Regression:")
print(f"  Logistic Regression Accuracy: {accuracy_log*100:.1f}%")
print(f"  KNN Accuracy: {accuracy_knn*100:.1f}%")
if accuracy_knn > accuracy_log:
    print(f"  ✓ KNN performs better by {(accuracy_knn - accuracy_log)*100:.1f} percentage points")
else:
    print(f"  ✓ Logistic Regression performs better by {(accuracy_log - accuracy_knn)*100:.1f} percentage points")

### 3.9 Confusion Matrices and Classification Metrics

Analyze model performance in detail using confusion matrix and other metrics.

In [ ]:
# Compute confusion matrices
cm_log = confusion_matrix(y_test, y_pred_log)
cm_knn = confusion_matrix(y_test, y_pred_knn)

print("\n" + "="*60)
print("CONFUSION MATRICES")
print("="*60)

print(f"\nLogistic Regression Confusion Matrix:")
print(cm_log)
print(f"\nKNN Confusion Matrix:")
print(cm_knn)

# Extract values for interpretation
tn_log, fp_log, fn_log, tp_log = cm_log.ravel()
tn_knn, fp_knn, fn_knn, tp_knn = cm_knn.ravel()

print(f"\n" + "="*60)
print("DETAILED CLASSIFICATION METRICS")
print("="*60)

print(f"\nLOGISTIC REGRESSION:")
print(classification_report(y_test, y_pred_log, target_names=['Answered', 'Refused']))

print(f"\nK-NEAREST NEIGHBORS:")
print(classification_report(y_test, y_pred_knn, target_names=['Answered', 'Refused']))

print(f"\nInterpretation Guide:")
print(f"  - Precision: Of people predicted to refuse, how many actually did?")
print(f"  - Recall: Of people who actually refused, how many did we identify?")
print(f"  - F1-Score: Balance between precision and recall")

### 3.10 Visualize Confusion Matrices and Model Performance

In [ ]:
# Visualize confusion matrices side by side
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(13, 5))

# Logistic Regression Confusion Matrix
sns.heatmap(cm_log, annot=True, fmt='d', cmap='Blues', ax=ax1, 
            xticklabels=['Answered', 'Refused'], yticklabels=['Answered', 'Refused'])
ax1.set_title('Logistic Regression Confusion Matrix', fontweight='bold', fontsize=12)
ax1.set_ylabel('Actual', fontweight='bold')
ax1.set_xlabel('Predicted', fontweight='bold')

# KNN Confusion Matrix
sns.heatmap(cm_knn, annot=True, fmt='d', cmap='Greens', ax=ax2,
            xticklabels=['Answered', 'Refused'], yticklabels=['Answered', 'Refused'])
ax2.set_title('K-Nearest Neighbors Confusion Matrix', fontweight='bold', fontsize=12)
ax2.set_ylabel('Actual', fontweight='bold')
ax2.set_xlabel('Predicted', fontweight='bold')

plt.tight_layout()
plt.show()

print("\nConfusion Matrix Interpretation:")
print("  - Top-left (True Negatives): Correctly predicted answered questions")
print("  - Top-right (False Positives): Incorrectly predicted refusal (false alarm)")
print("  - Bottom-left (False Negatives): Incorrectly predicted answered (missed refusers)")
print("  - Bottom-right (True Positives): Correctly predicted refusers")

### 3.11 Feature Importance Analysis

Determine which factors most strongly predict income question refusal.

In [ ]:
# Extract coefficients from logistic regression
coefficients = pd.DataFrame({
    'Feature': X.columns,
    'Coefficient': log_reg.coef_[0],
    'Abs_Coefficient': np.abs(log_reg.coef_[0])
}).sort_values('Abs_Coefficient', ascending=False)

print("\n" + "="*60)
print("FEATURE IMPORTANCE: What Predicts Income Refusal?")
print("="*60)
print(coefficients.to_string(index=False))

# Visualize feature importance
fig, ax = plt.subplots(figsize=(10, 6))
colors = ['red' if x < 0 else 'green' for x in coefficients['Coefficient']]
ax.barh(coefficients['Feature'], coefficients['Coefficient'], color=colors, alpha=0.7, edgecolor='black')
ax.set_xlabel('Logistic Regression Coefficient', fontweight='bold')
ax.set_title('Feature Importance for Predicting Income Question Refusal', fontweight='bold', fontsize=12)
ax.axvline(x=0, color='black', linestyle='--', linewidth=1)
plt.tight_layout()
plt.show()

print("\nInterpretation:")
print("  - Green bars (positive): Increase probability of refusing income questions")
print("  - Red bars (negative): Decrease probability of refusing income questions")
print("  - Magnitude (length): Strength of effect")

### 3.12 Model Prediction Examples

Show what the model predicts for different respondent profiles.

In [ ]:
# Create example respondent profiles
example_profiles = pd.DataFrame({
    'age': [25, 45, 65, 35, 50],
    'home_owner': [0, 1, 1, 0, 1],
    'education_Unknown': [0, 0, 0, 1, 0]  # Adjust based on your education categories
})

# Ensure columns match training data
for col in X.columns:
    if col not in example_profiles.columns:
        example_profiles[col] = 0

example_profiles = example_profiles[X.columns]

# Scale example profiles
example_scaled = scaler.transform(example_profiles)

# Get predictions and probabilities
example_predictions = log_reg.predict(example_scaled)
example_probabilities = log_reg.predict_proba(example_scaled)

results = pd.DataFrame({
    'Age': [25, 45, 65, 35, 50],
    'Home Owner': ['No', 'Yes', 'Yes', 'No', 'Yes'],
    'Prediction': ['Answers' if p == 0 else 'Refuses' for p in example_predictions],
    'Refusal Probability': [f"{prob[1]:.1%}" for prob in example_probabilities],
    'Confidence': [f"{max(prob)*100:.1f}%" for prob in example_probabilities]
})

print("\n" + "="*60)
print("MODEL PREDICTIONS FOR EXAMPLE RESPONDENT PROFILES")
print("="*60)
print(results.to_string(index=False))

print("\nKey Insight:")
print("Does higher education reduce income question refusal?")
print("Does home ownership affect willingness to share income information?")

---
# SUMMARY AND CONCLUSIONS

### Key Findings Across All Three Parts

In [ ]:
print("\n" + "="*80)
print("PROJECT SUMMARY: SILENCE CULTURE IN PUBLIC ATTITUDES")
print("="*80)

print("\n" + "─"*80)
print("PART 1: GLOBAL SILENCE HIERARCHY")
print("─"*80)
print("\nKey Finding:")
if not silence_df.empty:
    print(f"  Most taboo topic: {silence_df.iloc[0]['Topic']}")
    print(f"  Easiest to discuss: {silence_df.iloc[-1]['Topic']}")
    print(f"\nInterpretation:")
    print(f"  Silence varies significantly by topic (Mean={silence_df['Silence Score'].mean():.1f}%, SD={silence_df['Silence Score'].std():.1f}%)")
    print(f"  Some topics are universally discussed, others have strong taboos")

print("\n" + "─"*80)
print("PART 2: GENDER ROLE DISCUSSION PATTERNS BY COUNTRY")
print("─"*80)
print("\nKey Finding:")
if not country_refusal_df.empty:
    print(f"  Countries vary in willingness to discuss gender roles")
    print(f"  Avoidance range: {country_refusal_df['Avoidance Rate (%)'].min():.1f}% to {country_refusal_df['Avoidance Rate (%)'].max():.1f}%")
    print(f"\nInterpretation:")
    print(f"  Cultural differences in how openly gender issues are discussed")
    print(f"  Some societies show high conformity with high silence (social pressure)")
    print(f"  Others show disagreement suggesting diverse viewpoints")

print("\n" + "─"*80)
print("PART 3: PREDICTING INCOME QUESTION REFUSAL")
print("─"*80)
print(f"\nKey Finding:")
print(f"  Logistic Regression Accuracy: {accuracy_log*100:.1f}%")
print(f"  KNN Accuracy: {accuracy_knn*100:.1f}%")
print(f"\nInterpretation:")
print(f"  Education, age, and home ownership predict income disclosure behavior")
if not coefficients.empty:
    top_predictor = coefficients.iloc[0]
    print(f"  Strongest predictor: {top_predictor['Feature']}")
print(f"  Some people systematically avoid income questions regardless of context")

print("\n" + "="*80)
print("OVERALL CONCLUSION")
print("="*80)
print("""
Public willingness to discuss sensitive topics varies significantly:
1. By TOPIC - Some issues are universally taboo, others openly discussed
2. By COUNTRY - Cultural norms shape discussion comfort levels  
3. By INDIVIDUAL - Demographic factors predict information disclosure

This 'silence culture' has implications for:
- Survey research quality and missing data patterns
- Social policy: Which issues are politically difficult to address?
- Social cohesion: Do silences reflect consensus or suppressed conflict?
""")

print("="*80)

---
# Part 2: Advanced Analysis of the Culture of Silence

## 4. "Middle-Class Shame": The New Economic Silence

Goal:
- Measure the correlation between **self-defined social class** (Q158) and **missing / refusal in income questions** (Q173–Q174).
- Compare this relationship across years (2002 vs 2012 vs 2022).
- Hypothesis: Today (2022), low self-classed people (e.g. “lower class”, “working class”) are **more likely** to refuse income questions than in 2002, because poverty is seen more as a personal failure than fate.


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from scipy.stats import pearsonr, t
from sklearn.metrics import roc_curve, auc

sns.set_style("whitegrid")
plt.rcParams["figure.figsize"] = (10, 6)

# Load data for three years
# IMPORTANT: update filenames to your actual paths
df_2002 = pd.read_csv("2002Data.csv")
df_2012 = pd.read_csv("2012Data.csv")
df_2022 = pd.read_csv("2022Data.csv")

print("2002 shape:", df_2002.shape)
print("2012 shape:", df_2012.shape)
print("2022 shape:", df_2022.shape)


### 4.1 Construct social class and income refusal indicators

We need:
- `social_class_numeric`: a numeric scale for self-reported social class (Q158).
- `income_refusal`: binary indicator:
  - 1 = refused / missing in both Q173 and Q174
  - 0 = answered at least one income question

The exact text categories may differ by year/dataset, so the mapping here is illustrative and should be adjusted to your codes.


In [ ]:
def map_social_class_to_numeric(series):
    """
    Map self-reported social class (Q158) to a numeric scale.
    Example mapping:
        "lower class"          -> 1
        "working class"        -> 2
        "lower middle class"   -> 3
        "middle class"         -> 4
        "upper middle class"   -> 5
        "upper class"          -> 6

    IMPORTANT:
    - Replace keys in mapping with the actual labels in your dataset.
    """
    mapping = {
        "lower class": 1,
        "working class": 2,
        "lower middle class": 3,
        "middle class": 4,
        "upper middle class": 5,
        "upper class": 6
    }
    series_str = series.astype(str).str.lower()
    mapped = series_str.map(mapping)
    return mapped


def build_income_refusal(df, q173_col="Q173", q174_col="Q174"):
    """
    Build a binary variable 'income_refusal':
    - 1 if respondent refused or missing in BOTH Q173 and Q174
    - 0 otherwise.

    'Refusal' is detected via:
    - NaN
    - textual responses containing 'refuse', 'don\'t know', 'cannot choose', etc.

    IMPORTANT:
    - Update column names (q173_col, q174_col) to match your dataset.
    - Update refusal_keywords if your labels are coded differently.
    """
    refusal_keywords = ["refuse", "don’t know", "don't know", "cannot choose", "cannot decide"]

    def is_refusal(value):
        if pd.isna(value):
            return True
        text = str(value).lower()
        return any(kw in text for kw in refusal_keywords)

    # Initialize with zeros
    income_refusal = pd.Series(0, index=df.index, dtype=int)

    for idx, row in df.iterrows():
        val_173 = row[q173_col] if q173_col in df.columns else np.nan
        val_174 = row[q174_col] if q174_col in df.columns else np.nan

        if is_refusal(val_173) and is_refusal(val_174):
            income_refusal.loc[idx] = 1

    return income_refusal


### 4.2 Compute correlation between class and income refusal (per year)

We now:
1. Create `social_class_numeric` from Q158.
2. Create `income_refusal` from Q173–Q174.
3. Compute **Pearson correlation** between these two variables.
4. Build **95% confidence intervals** for the correlation using a t-based approach.

We will repeat this for 2002, 2012, 2022 and compare.


In [ ]:
def prepare_class_and_refusal(df,
                              class_col="Q158",
                              q173_col="Q173",
                              q174_col="Q174",
                              year_label="unknown"):
    """
    Prepare a clean DataFrame with:
    - social_class_numeric
    - income_refusal

    Returns a DataFrame with no missing in these two variables.
    """
    # Social class mapping
    if class_col not in df.columns:
        raise ValueError(f"{class_col} not found in {year_label} dataset")

    social_class_numeric = map_social_class_to_numeric(df[class_col])

    # Income refusal
    income_refusal = build_income_refusal(df, q173_col=q173_col, q174_col=q174_col)

    tmp = pd.DataFrame({
        "social_class_numeric": social_class_numeric,
        "income_refusal": income_refusal
    })

    # Drop rows with missing social class
    tmp_clean = tmp.dropna(subset=["social_class_numeric"])

    print(f"{year_label}: usable rows = {len(tmp_clean)}")
    return tmp_clean


def correlation_with_ci(x, y, alpha=0.05):
    """
    Compute Pearson correlation + (1-alpha)% CI using Fisher z-transform.
    """
    r, p_value = pearsonr(x, y)
    n = len(x)

    if n <= 3:
        return r, p_value, (np.nan, np.nan)

    # Fisher z
    z = np.arctanh(r)
    se = 1 / np.sqrt(n - 3)
    z_crit = t.ppf(1 - alpha/2, n - 3)  # approximate with t

    z_low = z - z_crit * se
    z_high = z + z_crit * se

    r_low = np.tanh(z_low)
    r_high = np.tanh(z_high)

    return r, p_value, (r_low, r_high)


In [ ]:
# IMPORTANT: Update column names if different in 2002/2012/2022
data_2002 = prepare_class_and_refusal(df_2002,
                                      class_col="Q158",
                                      q173_col="Q173",
                                      q174_col="Q174",
                                      year_label="2002")

data_2012 = prepare_class_and_refusal(df_2012,
                                      class_col="Q158",
                                      q173_col="Q173",
                                      q174_col="Q174",
                                      year_label="2012")

data_2022 = prepare_class_and_refusal(df_2022,
                                      class_col="Q158",
                                      q173_col="Q173",
                                      q174_col="Q174",
                                      year_label="2022")

# Compute correlations with CI
results_corr = []

for year, data in [("2002", data_2002), ("2012", data_2012), ("2022", data_2022)]:
    r, p, (low, high) = correlation_with_ci(data["social_class_numeric"],
                                            data["income_refusal"])
    results_corr.append({
        "year": year,
        "r": r,
        "p_value": p,
        "ci_low": low,
        "ci_high": high
    })

corr_df = pd.DataFrame(results_corr)
print("Correlation between social class and income refusal:")
print(corr_df)


### 4.3 Interpretation and visualization

- If **r > 0**: higher self-class → **more** refusal to report income.
- If **r < 0**: higher self-class → **less** refusal (i.e., lower classes are more silent).
- If |r| increases over time, the relationship between class and silence is strengthening.

We also visualize the correlation per year with error bars (CI).


In [ ]:
fig, ax = plt.subplots()

ax.errorbar(corr_df["year"],
            corr_df["r"],
            yerr=[corr_df["r"] - corr_df["ci_low"],
                  corr_df["ci_high"] - corr_df["r"]],
            fmt="o",
            capsize=5,
            color="darkred")

ax.axhline(0, color="black", linestyle="--", linewidth=1)
ax.set_xlabel("Year")
ax.set_ylabel("Pearson r (class vs income refusal)")
ax.set_title("Correlation Between Self-Class and Income Silence Over Time")

plt.show()

for _, row in corr_df.iterrows():
    print(f"Year {row['year']}: r={row['r']:.3f}, 95% CI=({row['ci_low']:.3f}, {row['ci_high']:.3f}), p={row['p_value']:.4f}")


## 5. "Political Spiral of Silence": Are We More Suspicious Now?

Goal:
- Build a **logistic regression classifier** that predicts who refuses to answer the “which party did you vote for” question (Q147).
- Features:
  - Region (center/periphery) – from place of residence (e.g., `region` or derived from locality code)
  - Age
  - Religion / religiosity (e.g., Q143–Q145)
- Compare model performance (especially **accuracy, ROC, AUC**) between:
  - 2012 data
  - 2022 data

Hypothesis:
- In 2022, political climate is more polarized and suspicious.
- The **accuracy** of the model in 2022 is higher → it is easier to predict “political silencers” using demographic profiles.


### 5.1 Construct target: political refusal (Q147)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (accuracy_score, confusion_matrix,
                             classification_report, roc_curve, auc)

def build_political_refusal(df, q147_col="Q147"):
    """
    Build binary variable 'political_refusal':
    - 1 if respondent refused / 'don't know' / 'cannot choose' / missing for Q147 (party voted).
    - 0 if they provided a valid party answer.

    IMPORTANT: Update refusal keywords and party coding as needed.
    """
    refusal_keywords = ["refuse", "don’t know", "don't know", "cannot choose", "cannot decide", "blank"]

    def is_refusal(value):
        if pd.isna(value):
            return True
        text = str(value).lower()
        # if it contains generic refusal / don't know text -> refusal
        if any(kw in text for kw in refusal_keywords):
            return True
        # Otherwise we treat it as a party choice
        return False

    target = df[q147_col].apply(is_refusal).astype(int)
    return target


### 5.2 Feature construction

We create:
- `region` (categorical): center / periphery / other.
  - This can be derived from locality or pre-coded region column.
- `age` (numerical): from year of birth or direct age.
- `religion` or `religiosity` (categorical): from Q143 / Q144 / Q145.

Because the exact column names and coding differ, we use placeholder names:
- `region_col = "region"`
- `age_col = "age"`
- `religion_col = "religion"`

You can replace them with the correct column names in your data.


In [ ]:
def prepare_political_dataset(df,
                              year_label="unknown",
                              q147_col="Q147",
                              region_col="region",
                              age_col="age",
                              religion_col="religion"):
    """
    Prepare X, y for political refusal classification.

    - y: political_refusal
    - X: region (categorical), age (numeric), religion (categorical)

    IMPORTANT:
    - Make sure region_col, age_col, religion_col exist or derive them before calling this function.
    """
    # Target
    if q147_col not in df.columns:
        raise ValueError(f"{q147_col} not found in {year_label} dataset")

    y = build_political_refusal(df, q147_col=q147_col)

    # Features (create minimal working example, you can enrich later)
    X = pd.DataFrame(index=df.index)

    # Age
    if age_col in df.columns:
        X["age"] = pd.to_numeric(df[age_col], errors="coerce")
    else:
        # If you only have year of birth, you can compute age:
        # X["age"] = 2022 - pd.to_numeric(df["birth_year"], errors="coerce")
        X["age"] = np.nan

    # Region
    if region_col in df.columns:
        X["region"] = df[region_col].astype(str)
    else:
        # Dummy fallback: center vs periphery based on locality if available
        # For now use placeholder
        X["region"] = "unknown"

    # Religion / religiosity
    if religion_col in df.columns:
        X["religion"] = df[religion_col].astype(str)
    else:
        X["religion"] = "unknown"

    # Drop rows with missing age (for simplicity)
    mask_valid = X["age"].notna()
    X = X[mask_valid]
    y = y[mask_valid]

    print(f"{year_label}: usable rows for political model = {len(X)}")
    return X, y


### 5.3 Train and evaluate logistic regression (2012 vs 2022)

We will:
- Split into train/test (e.g. 80/20, stratified).
- Use a **pipeline**:
  - ColumnTransformer:
    - scale numeric features (`age`)
    - one-hot encode categorical features (`region`, `religion`)
  - LogisticRegression classifier
- Compute:
  - Accuracy
  - Confusion matrix
  - Classification report
  - ROC curve and AUC


In [ ]:
def train_logistic_for_year(df, year_label,
                            q147_col="Q147",
                            region_col="region",
                            age_col="age",
                            religion_col="religion"):
    X, y = prepare_political_dataset(df,
                                     year_label=year_label,
                                     q147_col=q147_col,
                                     region_col=region_col,
                                     age_col=age_col,
                                     religion_col=religion_col)

    # Define feature types
    numeric_features = ["age"]
    categorical_features = ["region", "religion"]

    preprocessor = ColumnTransformer(
        transformers=[
            ("num", StandardScaler(), numeric_features),
            ("cat", OneHotEncoder(handle_unknown="ignore"), categorical_features)
        ]
    )

    clf = Pipeline(steps=[
        ("preprocessor", preprocessor),
        ("model", LogisticRegression(max_iter=1000, random_state=42))
    ])

    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, stratify=y, random_state=42
    )

    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    y_proba = clf.predict_proba(X_test)[:, 1]

    acc = accuracy_score(y_test, y_pred)
    cm = confusion_matrix(y_test, y_pred)
    report = classification_report(y_test, y_pred, target_names=["Answered", "Refused"])

    fpr, tpr, thresholds = roc_curve(y_test, y_proba)
    roc_auc = auc(fpr, tpr)

    results = {
        "year": year_label,
        "accuracy": acc,
        "confusion_matrix": cm,
        "classification_report": report,
        "fpr": fpr,
        "tpr": tpr,
        "roc_auc": roc_auc
    }
    return results


In [ ]:
# Train for 2012 and 2022
results_2012 = train_logistic_for_year(df_2012, "2012",
                                       q147_col="Q147",
                                       region_col="region",
                                       age_col="age",
                                       religion_col="religion")

results_2022 = train_logistic_for_year(df_2022, "2022",
                                       q147_col="Q147",
                                       region_col="region",
                                       age_col="age",
                                       religion_col="religion")

print("2012 accuracy:", results_2012["accuracy"])
print("2022 accuracy:", results_2022["accuracy"])


### 5.4 Compare metrics & plot ROC curves

In [ ]:
# Print detailed reports
for res in [results_2012, results_2022]:
    print("=" * 60)
    print(f"Year {res['year']} – Logistic Regression Performance")
    print("=" * 60)
    print("Accuracy:", f"{res['accuracy']:.3f}")
    print("\nConfusion matrix:\n", res["confusion_matrix"])
    print("\nClassification report:\n", res["classification_report"])

# Plot ROC curves
plt.figure()
plt.plot(results_2012["fpr"], results_2012["tpr"],
         label=f"2012 (AUC = {results_2012['roc_auc']:.3f})")
plt.plot(results_2022["fpr"], results_2022["tpr"],
         label=f"2022 (AUC = {results_2022['roc_auc']:.3f})")

plt.plot([0, 1], [0, 1], "k--", label="Random baseline")

plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curve – Political Refusal Classifier (2012 vs 2022)")
plt.legend(loc="lower right")
plt.grid(True, alpha=0.3)
plt.show()

print("\nInterpretation:")
print("- Higher AUC indicates better discrimination between 'answer' and 'refuse'.")
print("- If 2022 AUC >> 2012 AUC, it suggests that demographic profile")
print("  is more strongly linked to political silence in 2022.")


## 6. Polarization: Has the Distribution Changed?

Goal:
- Check whether Israelis’ opinions have become more **polarized** (fewer in the middle, more at the extremes).
- Use:
  - Histograms and kernel density plots
  - Standard deviation (σ) as a simple spread measure
- Questions:
  - Q74–Q77 (family values)
  - Q121 (women in politics)

Approach:
1. Recode responses into a numerical Likert scale:
   - Strongly disagree = -2
   - Disagree = -1
   - Neither = 0
   - Agree = 1
   - Strongly agree = 2
2. Plot distributions for 2002 vs 2022.
3. Compare:
   - shape (bell-shaped vs bi-modal)
   - standard deviation (higher σ → more spread / polarization).


### 6.1 Helper: Likert mapping



In [ ]:
def likert_to_numeric(series):
    """
    Map Likert-scale responses to numeric values.

    Example:
        'strongly disagree' -> -2
        'disagree'          -> -1
        'neither ...'       -> 0
        'agree'             -> 1
        'strongly agree'    -> 2

    IMPORTANT:
    - Update keywords according to actual language/labels in the data.
    - For Hebrew labels, you may map by code values instead.
    """
    mapping = {
        "strongly disagree": -2,
        "disagree": -1,
        "neither agree nor disagree": 0,
        "agree": 1,
        "strongly agree": 2
    }

    def map_single(v):
        if pd.isna(v):
            return np.nan
        text = str(v).lower()
        for key, val in mapping.items():
            if key in text:
                return val
        # If not matched, treat as missing
        return np.nan

    return series.apply(map_single)


### 6.2 Extract numeric responses for Q74–Q77 and Q121

We will:
- Convert each question to numeric (using `likert_to_numeric`).
- Compute:
  - mean
  - standard deviation
- Plot histograms + density curves for 2002 vs 2022.


In [ ]:
family_questions = ["Q74", "Q75", "Q76", "Q77"]
politics_question = "Q121"

def prepare_likert_for_year(df, questions, year_label):
    out = {}
    for q in questions:
        if q in df.columns:
            numeric = likert_to_numeric(df[q])
            out[q] = numeric.dropna()
            print(f"{year_label}: {q} – n={len(out[q])}, mean={out[q].mean():.2f}, std={out[q].std():.2f}")
        else:
            print(f"{year_label}: {q} not found in dataset")
    return out

print("=== 2002 Family & Politics ===")
likert_2002_family = prepare_likert_for_year(df_2002, family_questions, "2002")
likert_2002_politics = prepare_likert_for_year(df_2002, [politics_question], "2002")

print("\n=== 2022 Family & Politics ===")
likert_2022_family = prepare_likert_for_year(df_2022, family_questions, "2022")
likert_2022_politics = prepare_likert_for_year(df_2022, [politics_question], "2022")


### 6.3 Plot distributions: example for Q74 and Q121

In [ ]:
def plot_distribution_comparison(series_2002, series_2022, question_label, title_suffix=""):
    """
    Plot histogram + KDE for 2002 vs 2022 for a given question.
    """
    plt.figure(figsize=(10, 5))

    sns.histplot(series_2002, bins=5, kde=True, stat="density",
                 color="steelblue", alpha=0.5, label="2002")
    sns.histplot(series_2022, bins=5, kde=True, stat="density",
                 color="darkred", alpha=0.5, label="2022")

    plt.xlabel("Opinion (Likert numeric: -2 .. 2)")
    plt.ylabel("Density")
    plt.title(f"Distribution of Responses – {question_label} {title_suffix}")
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.show()

    print(f"{question_label} – 2002: mean={series_2002.mean():.2f}, std={series_2002.std():.2f}")
    print(f"{question_label} – 2022: mean={series_2022.mean():.2f}, std={series_2022.std():.2f}")
    print("Higher std suggests more spread / potential polarization.\n")


# Example: Q74 (family values) if present in both years
if "Q74" in likert_2002_family and "Q74" in likert_2022_family:
    plot_distribution_comparison(likert_2002_family["Q74"],
                                 likert_2022_family["Q74"],
                                 question_label="Q74 – Family Value Statement")

# Example: Q121 (women in politics) if present
q121_2002 = likert_2002_politics.get("Q121")
q121_2022 = likert_2022_politics.get("Q121")

if q121_2002 is not None and q121_2022 is not None:
    plot_distribution_comparison(q121_2002, q121_2022,
                                 question_label="Q121 – Women in Politics")


### 6.4 Optional: Aggregate polarization index across questions

In [ ]:
def polarization_summary(likert_dict, label):
    """
    Compute an aggregated polarization summary (average std across questions).
    """
    stds = []
    for q, s in likert_dict.items():
        if len(s) > 0:
            stds.append(s.std())
    if len(stds) == 0:
        print(f"{label}: no data")
        return
    print(f"{label}: average std across questions = {np.mean(stds):.3f}")

print("Family values – polarization summary")
polarization_summary(likert_2002_family, "2002 family")
polarization_summary(likert_2022_family, "2022 family")

print("\nPolitics (Q121) – single-question std comparison")
if q121_2002 is not None and q121_2022 is not None:
    print(f"2002: std={q121_2002.std():.3f}")
    print(f"2022: std={q121_2022.std():.3f}")
